<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
<center>Автор материала: программист-исследователь Mail.Ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ [Юрий Кашницкий](https://yorko.github.io/)

# <center> Capstone проект №1 <br> Идентификация пользователей по посещенным веб-страницам
<img src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'>

# <center>Неделя 6.  Vowpal Wabbit

На этой неделе мы познакомимся с популярной библиотекой Vowpal Wabbit и попробуем ее на данных по посещению сайтов.

**План 6 недели:**
- Часть 1. Статья по Vowpal Wabbit
- Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов
 - 2.1. Подготовка данных
 - 2.2. Валидация по отложенной выборке
 - 2.3. Валидация по тестовой выборке (Public Leaderboard)

**В этой части проекта Вам могут быть полезны видеозаписи следующих лекций курса "Обучение на размеченных данных":**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. `sklearn.linear_model`. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)
   
Также будет полезна [презентация](https://github.com/esokolov/ml-course-msu/blob/master/ML15/lecture-notes/Sem08_vw.pdf) лектора специализации Евгения Соколова. И, конечно же, [документация](https://github.com/JohnLangford/vowpal_wabbit/wiki) Vowpal Wabbit.

### Задание
1. Заполните код в этой тетрадке 
2. Если вы проходите специализацию Яндеса и МФТИ, пошлите файл с ответами в соответствующем Programming Assignment. <br> Если вы проходите курс ODS, выберите ответы в [веб-форме](https://docs.google.com/forms/d/1wteunpEhAt_9s-WBwxYphB6XpniXsAZiFSNuFNmvOdk).

## Часть 1. Статья про Vowpal Wabbit
Прочитайте [статью](https://habrahabr.ru/company/ods/blog/326418/) про Vowpal Wabbit на Хабре из серии открытого курса OpenDataScience по машинному обучению. Материал для этой статьи зародился из нашей специализации. Скачайте [тетрадку](https://github.com/Yorko/mlcourse_open/blob/master/jupyter_russian/topic08_sgd_hashing_vowpal_wabbit/topic8_sgd_hashing_vowpal_wabbit.ipynb), прилагаемую к статье, посмотрите код, изучите его, поменяйте, только так можно разобраться с Vowpal Wabbit.

## Часть 2. Применение Vowpal Wabbit к данным по посещению сайтов

### 2.1. Подготовка данных

**Далее посмотрим на Vowpal Wabbit в деле. Правда, в задаче нашего соревнования при бинарной классификации веб-сессий мы разницы не заметим – как по качеству, так и по скорости работы (хотя можете проверить), продемонстрируем всю резвость VW в задаче классификации на 400 классов. Исходные данные все те же самые, но выделено 400 пользователей, и решается задача их идентификации. Скачайте данные [отсюда](https://kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/data) – файлы `train_sessions_400users.csv` и `test_sessions_400users.csv`.**

In [1]:
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.linear_model import LogisticRegression, SGDClassifier

In [2]:
# Поменяйте на свой путь к данным
PATH_TO_DATA = 'capstone_user_identification'

**Загрузим обучающую и тестовую выборки. Можете заметить, что тестовые сессии здесь по времени четко отделены от сессий в обучающей выборке. **

In [3]:
train_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'train_sessions_400users.csv'), 
                           index_col='session_id')

In [4]:
test_df_400 = pd.read_csv(os.path.join(PATH_TO_DATA,'test_sessions_400users.csv'), 
                           index_col='session_id')

In [5]:
train_df_400

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,user_id
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,2014-03-24 15:22:55,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,2014-04-17 14:26:01,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,2014-03-21 10:13:36,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,2013-12-13 09:54:34,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,2013-11-26 12:35:32,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182789,614,2014-01-17 15:36:09,1340.0,2014-01-17 15:36:12,109.0,2014-01-17 15:36:39,108.0,2014-01-17 15:36:39,109.0,2014-01-17 15:36:40,...,2014-01-17 15:36:48,9.0,2014-01-17 15:36:52,1340.0,2014-01-17 15:38:03,615.0,2014-01-17 15:38:22,612.0,2014-01-17 15:39:03,183
182790,8,2013-11-26 10:34:19,7.0,2013-11-26 10:34:20,8.0,2013-11-26 10:34:20,7.0,2013-11-26 10:34:24,17.0,2013-11-26 10:34:32,...,2013-11-26 10:34:35,7.0,2013-11-26 10:34:36,11.0,2013-11-26 10:34:38,11.0,2013-11-26 10:34:52,7.0,2013-11-26 10:34:57,448
182791,7,2014-02-26 13:45:34,10.0,2014-02-26 13:45:34,11.0,2014-02-26 13:45:34,8.0,2014-02-26 13:51:02,8.0,2014-02-26 13:51:03,...,2014-02-26 13:51:04,8.0,2014-02-26 13:51:04,8.0,2014-02-26 13:51:08,8.0,2014-02-26 13:51:11,8.0,2014-02-26 13:51:13,632


**Видим, что в обучающей выборке 182793 сессий, в тестовой – 46473, и сессии действительно принадлежат 400 различным пользователям.**

In [6]:
train_df_400.shape, test_df_400.shape, train_df_400['user_id'].nunique()

((182793, 21), (46473, 20), 400)

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 400). Поэтому придется применить `LabelEncoder`, да еще и +1 потом добавить (`LabelEncoder` переводит метки в диапозон от 0 до K-1). Потом надо будет применить обратное преобразование.**

In [7]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [8]:
y = train_df_400['user_id']
class_encoder = LabelEncoder()
y_for_vw = pd.Series(class_encoder.fit_transform(y))+1
y_for_vw.index += 1

In [9]:
train_df_400['user_id_vw'] = y_for_vw

In [10]:
train_df_400

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,site7,time7,site8,time8,site9,time9,site10,time10,user_id,user_id_vw
session_id,,,,,,,,,,,,,,,,,,,,,
1,23713,2014-03-24 15:22:40,23720.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:48,23713.0,2014-03-24 15:22:54,23720.0,2014-03-24 15:22:54,...,23713.0,2014-03-24 15:23:01,23713.0,2014-03-24 15:23:03,23713.0,2014-03-24 15:23:04,23713.0,2014-03-24 15:23:05,653,262
2,8726,2014-04-17 14:25:58,8725.0,2014-04-17 14:25:59,665.0,2014-04-17 14:25:59,8727.0,2014-04-17 14:25:59,45.0,2014-04-17 14:25:59,...,45.0,2014-04-17 14:26:01,5320.0,2014-04-17 14:26:18,5320.0,2014-04-17 14:26:47,5320.0,2014-04-17 14:26:48,198,82
3,303,2014-03-21 10:12:24,19.0,2014-03-21 10:12:36,303.0,2014-03-21 10:12:54,303.0,2014-03-21 10:13:01,303.0,2014-03-21 10:13:24,...,303.0,2014-03-21 10:13:54,309.0,2014-03-21 10:14:01,303.0,2014-03-21 10:14:06,303.0,2014-03-21 10:14:24,34,16
4,1359,2013-12-13 09:52:28,925.0,2013-12-13 09:54:34,1240.0,2013-12-13 09:54:34,1360.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:54:34,...,1346.0,2013-12-13 09:54:34,1345.0,2013-12-13 09:54:34,1344.0,2013-12-13 09:58:19,1345.0,2013-12-13 09:58:19,601,241
5,11,2013-11-26 12:35:29,85.0,2013-11-26 12:35:31,52.0,2013-11-26 12:35:31,85.0,2013-11-26 12:35:32,11.0,2013-11-26 12:35:32,...,11.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:03,10.0,2013-11-26 12:37:03,85.0,2013-11-26 12:37:04,273,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182789,614,2014-01-17 15:36:09,1340.0,2014-01-17 15:36:12,109.0,2014-01-17 15:36:39,108.0,2014-01-17 15:36:39,109.0,2014-01-17 15:36:40,...,9.0,2014-01-17 15:36:52,1340.0,2014-01-17 15:38:03,615.0,2014-01-17 15:38:22,612.0,2014-01-17 15:39:03,183,74
182790,8,2013-11-26 10:34:19,7.0,2013-11-26 10:34:20,8.0,2013-11-26 10:34:20,7.0,2013-11-26 10:34:24,17.0,2013-11-26 10:34:32,...,7.0,2013-11-26 10:34:36,11.0,2013-11-26 10:34:38,11.0,2013-11-26 10:34:52,7.0,2013-11-26 10:34:57,448,180
182791,7,2014-02-26 13:45:34,10.0,2014-02-26 13:45:34,11.0,2014-02-26 13:45:34,8.0,2014-02-26 13:51:02,8.0,2014-02-26 13:51:03,...,8.0,2014-02-26 13:51:04,8.0,2014-02-26 13:51:08,8.0,2014-02-26 13:51:11,8.0,2014-02-26 13:51:13,632,254


In [11]:
test_df_400

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
session_id,,,,,,,,,,,,,,,,,,,,
1,9,2014-10-04 12:24:43,304.0,2014-10-04 12:25:34,308.0,2014-10-04 12:28:33,307.0,2014-10-04 12:28:33,91.0,2014-10-04 12:28:33,308.0,2014-10-04 12:28:34,312.0,2014-10-04 12:30:31,300.0,2014-10-04 12:31:30,305.0,2014-10-04 12:32:31,309.0,2014-10-04 12:34:31
2,838,2014-12-02 09:20:37,504.0,2014-12-02 09:20:38,68.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:38,838.0,2014-12-02 09:20:38,11.0,2014-12-02 09:20:40,838.0,2014-12-02 09:20:41,886.0,2014-12-02 09:20:42,27.0,2014-12-02 09:20:42,305.0,2014-12-02 09:20:42
3,190,2014-10-01 09:27:38,192.0,2014-10-01 09:27:38,8.0,2014-10-01 09:27:38,189.0,2014-10-01 09:27:38,191.0,2014-10-01 09:27:39,189.0,2014-10-01 09:27:39,190.0,2014-10-01 09:27:39,2375.0,2014-10-01 09:27:39,192.0,2014-10-01 09:27:39,8.0,2014-10-01 09:27:39
4,295,2014-10-02 14:34:17,295.0,2014-10-02 14:34:18,295.0,2014-10-02 14:34:19,295.0,2014-10-02 14:34:20,295.0,2014-10-02 14:34:21,295.0,2014-10-02 14:34:22,295.0,2014-10-02 14:34:23,295.0,2014-10-02 14:34:24,295.0,2014-10-02 14:34:25,295.0,2014-10-02 14:34:26
5,31,2014-05-19 17:50:21,3177.0,2014-05-19 17:50:21,3174.0,2014-05-19 17:50:21,32434.0,2014-05-19 17:50:22,27.0,2014-05-19 17:50:22,31.0,2014-05-19 17:50:22,5698.0,2014-05-19 17:50:25,5698.0,2014-05-19 17:50:26,5698.0,2014-05-19 17:50:27,4173.0,2014-05-19 17:50:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46469,6209,2014-05-28 12:16:00,308.0,2014-05-28 12:16:01,750.0,2014-05-28 12:16:01,4351.0,2014-05-28 12:16:01,6209.0,2014-05-28 12:16:31,6209.0,2014-05-28 12:16:32,27.0,2014-05-28 12:16:32,6209.0,2014-05-28 12:16:34,750.0,2014-05-28 12:16:36,6209.0,2014-05-28 12:16:41
46470,8,2014-08-04 17:30:01,982.0,2014-08-04 17:30:02,8.0,2014-08-04 17:30:06,982.0,2014-08-04 17:30:17,110.0,2014-08-04 17:31:33,8.0,2014-08-04 17:31:33,49.0,2014-08-04 17:31:35,11.0,2014-08-04 17:31:35,692.0,2014-08-04 17:31:35,325.0,2014-08-04 17:31:35
46471,295,2014-11-02 08:48:27,295.0,2014-11-02 08:48:28,295.0,2014-11-02 08:48:31,295.0,2014-11-02 08:48:32,295.0,2014-11-02 08:49:01,295.0,2014-11-02 08:49:03,295.0,2014-11-02 08:49:04,295.0,2014-11-02 08:49:13,294.0,2014-11-02 08:49:15,294.0,2014-11-02 08:49:16


**Далее будем сравнивать VW с SGDClassifier и с логистической регрессией. Всем моделям этим нужна предобработка входных данных. Подготовьте для sklearn-моделей разреженные матрицы, как мы это делали в 5 части:**
- объедините обучающиую и тестовую выборки
- выберите только сайты (признаки от 'site1' до 'site10')
- замените пропуски на нули (сайты у нас нумеровались с 0)
- переведите в разреженный формат `csr_matrix`
- разбейте обратно на обучающую и тестовую части

In [12]:
def func_csr_matrix(data_for_m):
    docs = data_for_m.iloc[:, :].values
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}

    for d in docs:
        for t in d:
            index = vocabulary.setdefault(t, t)
            indices.append(index)
            data.append(1)    
        indptr.append(len(indices))
    
    return csr_matrix((data, indices, indptr), dtype=int)[:, 1:]

In [13]:
sites = ['site' + str(i) for i in range(1, 11)]
train_test_df_400 = pd.concat([train_df_400, test_df_400])
train_test_df_sites = train_test_df_400[sites].fillna(0).astype('int')

In [14]:
train_test_sparse = func_csr_matrix(train_test_df_sites).astype('uint8')

In [15]:
X_train_sparse = train_test_sparse[:train_df_400.shape[0],:]
X_test_sparse = train_test_sparse[-test_df_400.shape[0]:,:]
y = train_df_400['user_id'].values

### 2.2. Валидация по отложенной выборке

**Выделим обучающую (70%) и отложенную (30%) части исходной обучающей выборки. Данные не перемешиваем, учитываем, что сессии отсортированы по времени.**

In [16]:
train_share = int(.7 * train_df_400.shape[0])
train_df_part = train_df_400[sites].iloc[:train_share, :]
valid_df = train_df_400[sites].iloc[train_share:, :]
X_train_part_sparse = X_train_sparse[:train_share, :]
X_valid_sparse = X_train_sparse[train_share:, :]

In [17]:
y_train_part = y[:train_share]
y_valid = y[train_share:]
y_train_part_for_vw = y_for_vw[:train_share]
y_valid_for_vw = y_for_vw[train_share:]

**Реализуйте функцию, `arrays_to_vw`, переводящую обучающую выборку в формат Vowpal Wabbit.**

Вход:
 - X – матрица `NumPy` (обучающая выборка)
 - y (необяз.) - вектор ответов (`NumPy`). Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - train – флаг, True в случае обучающей выборки, False – в случае тестовой выборки
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- надо пройтись по всем строкам матрицы `X` и записать через пробел все значения, предварительно добавив вперед нужную метку класса из вектора `y` и знак-разделитель `|`
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [18]:
def arrays_to_vw(X, y=None, train=True, out_file='tmp.vw'):
    X = np.nan_to_num(X)
    with open(out_file, 'w') as vw_data:
        if  y is not None and train == True:
            y = y.reshape(-1,1)
            X = np.append(X, y, axis=1)
            for t in X:
                vw_data.write(str(int(t[-1])) + ' | ' + ' '.join([str(int(elem)) for elem in t[:-1]]) + '\n')            
        else:
            for t in X:
                vw_data.write('1' + ' | ' + ' '.join([str(int(elem)) for elem in t]) + '\n')

**Примените написанную функцию к части обучащей выборки `(train_df_part, y_train_part_for_vw)`, к отложенной выборке `(valid_df, y_valid_for_vw)`, ко всей обучающей выборке и ко всей тестовой выборке. Обратите внимание, что на вход наш метод принимает именно матрицы и вектора `NumPy`.**

In [19]:
%%time
# будет 4 вызова
arrays_to_vw(train_df_part.values, y_train_part_for_vw.values, out_file = os.path.join(PATH_TO_DATA,'train_part.vw'))
arrays_to_vw(valid_df.values, y_valid_for_vw.values, out_file = os.path.join(PATH_TO_DATA,'valid.vw'))
arrays_to_vw(train_test_df_sites[:train_df_400.shape[0]].values, y_for_vw.values, out_file = os.path.join(PATH_TO_DATA,'train.vw'))
arrays_to_vw(X = train_test_df_sites[-test_df_400.shape[0]:].values, out_file = os.path.join(PATH_TO_DATA,'test.vw'))

Wall time: 2.78 s


**Результат должен получиться таким.**

In [20]:
!powershell -command "& {get-content capstone_user_identification\train_part.vw|select-object -first 3} 

262 | 23713 23720 23713 23713 23720 23713 23713 23713 23713 23713
82 | 8726 8725 665 8727 45 8725 45 5320 5320 5320
16 | 303 19 303 303 303 303 303 309 303 303


In [21]:
!powershell -command "& {get-content capstone_user_identification\valid.vw|select-object -first 3} 

4 | 7 923 923 923 11 924 7 924 838 7
160 | 91 198 11 11 302 91 668 311 310 91
312 | 27085 848 118 118 118 118 11 118 118 118


In [22]:
!powershell -command "& {get-content capstone_user_identification\test.vw|select-object -first 3} 

1 | 9 304 308 307 91 308 312 300 305 309
1 | 838 504 68 11 838 11 838 886 27 305
1 | 190 192 8 189 191 189 190 2375 192 8


**Обучите модель Vowpal Wabbitна выборке `train_part.vw`. Укажите, что решается задача классификации с 400 классами (`--oaa`), сделайте 3 прохода по выборке (`--passes`). Задайте некоторый кэш-файл (`--cache_file`, можно просто указать флаг `-c`), так VW будет быстрее делать все следующие после первого проходы по выборке (прошлый кэш-файл удаляется с помощью аргумента `-k`). Также укажите значение параметра `b`=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Наконец, укажите `random_seed`=17. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другие функции потерь.**

In [23]:
train_part_vw = os.path.join(PATH_TO_DATA, 'train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [24]:
from vowpalwabbit import pyvw

In [25]:
%%time
!vw --oaa 400 -d $PATH_TO_DATA\train_part.vw -f $PATH_TO_DATA\vw_model.vw --random_seed 17 --passes 3 -c -k -b 26

Wall time: 29.7 s


final_regressor = capstone_user_identification\vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = capstone_user_identification\train_part.vw.cache
Reading datafile = capstone_user_identification\train_part.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128          12

**Запишите прогнозы на выборке *valid.vw* в *vw_valid_pred.csv*.**

In [26]:
!vw -i $PATH_TO_DATA\vw_model.vw -t -d $PATH_TO_DATA\valid.vw \
-p $PATH_TO_DATA\vw_valid_pred.csv

only testing
predictions = capstone_user_identification\vw_valid_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = capstone_user_identification\valid.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        4      188       11
1.000000 1.000000            2            2.0      160      220       11
0.750000 0.500000            4            4.0      143      143       11
0.750000 0.750000            8            8.0      247      247       11
0.687500 0.625000           16           16.0      341       30       11
0.593750 0.500000           32           32.0      237      237       11
0.609375 0.625000           64           64.0      178      178       11
0.640625 0.671875          128          128.0      132      228       11
0.656250 0.671875          256         

**Считайте прогнозы *kaggle_data/vw_valid_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [27]:
from sklearn.metrics import classification_report, accuracy_score

In [28]:
with open(os.path.join(PATH_TO_DATA, 'vw_valid_pred.csv')) as pred_file:
    valid_prediction_mult = [int(label) 
                            for label in pred_file.readlines()]
with open(valid_vw) as valid_file:
    valid_lable_mult = [int(label[:label.find(' ')]) 
                            for label in valid_file.readlines()]

In [29]:
print("\n\nValid data")
print("Accuracy : %f" % accuracy_score(valid_lable_mult, valid_prediction_mult))
print("Y: ", valid_lable_mult[:10])
print("Prediction: ", valid_prediction_mult[:10])



Valid data
Accuracy : 0.345417
Y:  [4, 160, 312, 143, 319, 148, 131, 247, 25, 219]
Prediction:  [188, 220, 364, 143, 132, 350, 8, 247, 25, 219]


**Теперь обучите `SGDClassifier` (3 прохода по выборке, логистическая функция потерь) и `LogisticRegression` на 70% разреженной обучающей выборки – `(X_train_part_sparse, y_train_part)`, сделайте прогноз для отложенной выборки `(X_valid_sparse, y_valid)` и посчитайте доли верных ответов. Логистическая регрессия будет обучаться не быстро (у меня – 4 минуты) – это нормально. Укажите везде `random_state`=17, `n_jobs`=-1. Для `SGDClassifier` также укажите `max_iter=3`.**

In [30]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier

In [31]:
logit = LogisticRegression(random_state=17, n_jobs=-1,  solver='lbfgs')

In [32]:
%%time
logit.fit(X_train_part_sparse, y_train_part)

Wall time: 6min 52s


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [33]:
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17, max_iter=3)

In [34]:
%%time
sgd_logit.fit(X_train_part_sparse, y_train_part)

Wall time: 5.72 s


C:\Users\oorzhak\Anaconda3\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=3,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=17, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

**<font color='red'>Вопрос 1. </font> Посчитайте долю правильных ответов на отложенной выборке для Vowpal Wabbit, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 2. </font> Посчитайте долю правильных ответов на отложенной выборке для SGD, округлите до 3 знаков после запятой.**

**<font color='red'>Вопрос 3. </font> Посчитайте долю правильных ответов на отложенной выборке для логистической регрессии, округлите до 3 знаков после запятой.**

In [35]:
sgd_predictions = sgd_logit.predict(X_valid_sparse.toarray())

In [36]:
logit_predictions = logit.predict(X_valid_sparse.toarray())

In [37]:
vw_valid_acc = accuracy_score(valid_lable_mult, valid_prediction_mult)
sgd_valid_acc = accuracy_score(y_valid, sgd_predictions)
logit_valid_acc = accuracy_score(y_valid, logit_predictions)

In [38]:
def write_answer_to_file(answer, file_address):
    with open(file_address, 'w') as out_f:
        out_f.write(str(answer))

In [39]:
write_answer_to_file(round(vw_valid_acc, 3), 'answer6_1.txt')
write_answer_to_file(round(sgd_valid_acc, 3), 'answer6_2.txt')
write_answer_to_file(round(logit_valid_acc, 3), 'answer6_3.txt')

### 2.3. Валидация по тестовой выборке (Public Leaderboard)

**Обучите модель VW с теми же параметрами на всей обучающей выборке – *train.vw*.**

In [40]:
%%time
!vw --oaa 400 -d $PATH_TO_DATA\train.vw -f $PATH_TO_DATA\vw_model.vw --random_seed 17 --passes 3 -c -k -b 26

Wall time: 35.8 s


final_regressor = capstone_user_identification\vw_model.vw
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = capstone_user_identification\train.vw.cache
Reading datafile = capstone_user_identification\train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0      262        1       11
1.000000 1.000000            2            2.0       82      262       11
1.000000 1.000000            4            4.0      241      262       11
1.000000 1.000000            8            8.0      352      262       11
1.000000 1.000000           16           16.0      135       16       11
1.000000 1.000000           32           32.0       71      112       11
0.968750 0.937500           64           64.0      358      231       11
0.976563 0.984375          128          128.0      3

**Сделайте прогноз для тестовой выборки.**

In [41]:
%%time
!vw -i $PATH_TO_DATA\vw_model.vw -t -d $PATH_TO_DATA\test.vw -p $PATH_TO_DATA\vw_test_pred.csv

Wall time: 868 ms


only testing
predictions = capstone_user_identification\vw_test_pred.csv
Num weight bits = 26
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = capstone_user_identification\test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0        1       90       11
1.000000 1.000000            2            2.0        1       21       11
1.000000 1.000000            4            4.0        1      265       11
1.000000 1.000000            8            8.0        1      137       11
1.000000 1.000000           16           16.0        1      273       11
1.000000 1.000000           32           32.0        1      384       11
1.000000 1.000000           64           64.0        1      139       11
1.000000 1.000000          128          128.0        1       85       11
1.000000 1.000000          256          2

**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle.**

In [42]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [48]:
with open(os.path.join(PATH_TO_DATA, 'vw_test_pred.csv')) as pred_file:
    pred_lbls = np.array([int(label) 
                            for label in pred_file.readlines()])
pred_lbls = pred_lbls -1
vw_pred = class_encoder.inverse_transform(pred_lbls)

array([224,  48, 795, ..., 107, 387, 179], dtype=int64)

In [49]:
write_to_submission_file(vw_pred, os.path.join(PATH_TO_DATA, 'vw_400_users.csv'))

**Сделайте то же самое для SGD и логистической регрессии. Тут уже ждать обучение логистической регрессии совсем скучно (заново запускать тетрадку вам не захочется), но давайте дождемся.**

In [50]:
%%time
logit.fit(X_train_sparse, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=-1, penalty='l2', random_state=17,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [63]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
sgd_logit.fit(X_train_sparse, y)

Wall time: 18.3 s


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=-1, penalty='l2', power_t=0.5,
              random_state=17, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [64]:
sgd_logit_test_pred = sgd_logit.predict(X_test_sparse.toarray())

In [54]:
logit_test_pred = logit.predict(X_test_sparse.toarray())

In [65]:
write_to_submission_file(logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'logit_400_users.csv'))
write_to_submission_file(sgd_logit_test_pred, 
                         os.path.join(PATH_TO_DATA, 'sgd_400_users.csv'))

Посмотрим на доли правильных ответов на публичной части (public leaderboard) тестовой выборки [этого](https://www.kaggle.com/c/identify-me-if-you-can4/submit) соревнования.

**<font color='red'>Вопрос 4. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для Vowpal Wabbit?**

**<font color='red'>Вопрос 5. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для SGD?**

**<font color='red'>Вопрос 6. </font> Какова доля правильных ответов на публичной части тестовой выборки (public leaderboard)  для логистической регрессии?**


In [66]:
vw_lb_score, sgd_lb_score, logit_lb_score = 0.18768, 0.18316, 0.19289

write_answer_to_file(round(vw_lb_score, 3), 'answer6_4.txt')
write_answer_to_file(round(sgd_lb_score, 3), 'answer6_5.txt')
write_answer_to_file(round(logit_lb_score, 3), 'answer6_6.txt')

**В заключение по заданию:**
- Про соотношение качества классификации и скорости обучения VW, SGD и logit выводы предлагается сделать самостоятельно
- Пожалуй, задача классификации на 400 классов (идентификация 400 пользователей) решается недостаточно хорошо при честном отделении по времени тестовой выборки от обучающей. Далее мы будем соревноваться в идентификации одного пользователя (Элис) – [вот](https://kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнование, в котором предлагается поучаствовать. Не перепутайте! 

**Удачи!**